In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
#import xgboost as xgb

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectFromModel
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

We load the training set and the validation set which have been already preprocessed

In [3]:
#Importing dataset
train = pd.read_csv("../dataset/processed/train_rand.csv", index_col=0)
val1 = pd.read_csv("../dataset/processed/val1_rand.csv", index_col=0)
val2 = pd.read_csv("../dataset/processed/val2.csv", index_col=0)

In [4]:
train.OrdinalDate = train.OrdinalDate.map(lambda x: x % 365)
val1.OrdinalDate = val1.OrdinalDate.map(lambda x: x % 365)
val2.OrdinalDate = val2.OrdinalDate.map(lambda x: x % 365)

We prepare the training input samples and the labels for all the sets

In [5]:
X=train.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)
X_val1=val1.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)
X_val2=val2.drop(['StoreID','Date', 'NumberOfSales', 'Region'], axis=1)

y=train['NumberOfSales']
y_val1=val1['NumberOfSales']
y_val2=val2['NumberOfSales']

We try to find good hyperparameter in xGBoost. First of all we try to find good values for max_depth and colsample_bytree 

In [6]:
#depth_range = range()
#col_range = range()

for d in depth_range:
    for col in col_range:
            xclas = XGBRegressor(subsample=0.7, eta=0.3, max_depth=d, 
                                 colsample_bytree=col, nthread=4,
                                 min_child_weight=5, gamma=10,
                                 n_estimators=50)  
            xclas.fit(X, y)   
            y_pred = xclas.predict(X_val1) 
            mae_val1=mean_absolute_error(y_val1,y_pred)

            print("max_depth= ", d, " colsample_bytree= ", col, " mae= ", mae_val1)

NameError: name 'depth_range' is not defined

At the end predict the validation error on val2 and we compute the csv in which there is the result

In [10]:
xclas = XGBRegressor(subsample=0.6, eta=0.3, max_depth=17, 
                         colsample_bytree=0.7, nthread=4,
                         min_child_weight=5, gamma=0,
                         n_estimators=200)  

xclas.fit(X, y)  

y_pred_val2 = xclas.predict(X_val2) 
mae_val2=mean_absolute_error(y_val2,y_pred_val2)

print("mae_val2=",mae_val2)

mae_val2= 600.470504139


In [18]:
val2=val2.reset_index(drop=True)
y_val2 = y_val2.reset_index(drop=True)

columns=['Date', 'StoreID', 'RegionID', 'SalesPredicted', 'SalesReal']
index=range(y_val2.shape[0])
result=pd.DataFrame(index=index,columns=columns)

result['Date']=val2['Date']
result['StoreID']=val2['StoreID']
result['RegionID']=val2['Region']
result['SalesPredicted']=y_pred_val2
result['SalesReal']=y_val2

result.to_csv("results_xg.csv")